<a href="https://colab.research.google.com/github/nalgo-intern/team-a/blob/master/flow/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **機械学習**

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

前処理済みデータを共有リンクからダウンロード

In [0]:
downloaded = drive.CreateFile({'id':'1WxOK8oyxZ_jJkaJ4zLa_nbfnmDijh0M9'})

downloaded.GetContentFile('train_p_nothuman.zip')

In [0]:
downloaded = drive.CreateFile({'id':'1oiDVeU8A14ukiD247w8G_xueVhwiy2uK'})

downloaded.GetContentFile('validation_p_nothuman.zip')

In [0]:
!unzip train_p_nothuman.zip
!unzip validation_p_nothuman.zip

Archive:  train_p_nothuman.zip
replace train/chimpanzee/10037667855.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train/chimpanzee/10037667855.jpg  
  inflating: train/chimpanzee/10059377785.jpg  
  inflating: train/chimpanzee/10803455094.jpg  
  inflating: train/chimpanzee/11581412744.jpg  
  inflating: train/chimpanzee/11950809213.jpg  
  inflating: train/chimpanzee/12184235744.jpg  
  inflating: train/chimpanzee/14974695853.jpg  
  inflating: train/chimpanzee/14993392243.jpg  
  inflating: train/chimpanzee/15039218068.jpg  
  inflating: train/chimpanzee/15165394638.jpg  
  inflating: train/chimpanzee/15276624803.jpg  
  inflating: train/chimpanzee/15610867861.jpg  
  inflating: train/chimpanzee/15618200812.jpg  
  inflating: train/chimpanzee/15657269754.jpg  
  inflating: train/chimpanzee/15896275615.jpg  
  inflating: train/chimpanzee/16115811169.jpg  
  inflating: train/chimpanzee/16277952091.jpg  
  inflating: train/chimpanzee/16501300762.jpg  
  inflating: train/chim

### 転移学習

In [0]:
from keras.models import Model
from keras.layers import Dense,Input,GlobalMaxPooling2D,Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

N_CATEGORIES = 3
IMAGE_SIZE = 224
BATCH_SIZE = 8

NUM_TRAINING = 288
NUM_VALIDATION = 72

input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model = VGG16(weights='imagenet', include_top=False,input_tensor=input_tensor)

x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(.25)(x)
x = Dense(1024, activation='relu')(x)


predictions = Dense(N_CATEGORIES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers[:13]:
   layer.trainable = False

model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

json_string=model.to_json()
open("model"+'.json','w').write(json_string)

train_datagen = ImageDataGenerator(
   rescale=1.0 / 255,
   shear_range=0,
   zoom_range=0.1,
   horizontal_flip=True,
   rotation_range=0)

test_datagen = ImageDataGenerator(
   rescale=1.0 / 255,
)
train_generator = train_datagen.flow_from_directory(
   'train',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
   'validation',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

hist = model.fit_generator(train_generator,
   steps_per_epoch=NUM_TRAINING//BATCH_SIZE,
   epochs=80,
   verbose=1,
   validation_data=validation_generator,
   validation_steps=NUM_VALIDATION//BATCH_SIZE,
   )

model.save('ape.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

KeyboardInterrupt: ignored

結果を上げる

In [0]:
upload_file_2 = drive.CreateFile()
upload_file_2.SetContentFile("ape.hdf5")
upload_file_2.Upload()